<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1171875
Current benign train loss: 2.3601717948913574

Current batch: 100
Current benign train accuracy: 0.328125
Current benign train loss: 1.833861231803894

Current batch: 200
Current benign train accuracy: 0.375
Current benign train loss: 1.6179919242858887

Current batch: 300
Current benign train accuracy: 0.390625
Current benign train loss: 1.652824878692627

Total benign train accuarcy: 33.628
Total benign train loss: 717.0355360507965

[ Test epoch: 0 ]

Test accuarcy: 46.67
Test average loss: 0.014461997437477113
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.453125
Current benign train loss: 1.5518718957901

Current batch: 100
Current benign train accuracy: 0.4765625
Current benign train loss: 1.4004385471343994

Current batch: 200
Current benign train accuracy: 0.5703125
Current benign train loss: 1.3083237409591675

Current batch: 300
Current benign train accuracy: 0.


Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.33135733008384705

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.25320950150489807

Total benign train accuarcy: 87.926
Total benign train loss: 136.70063956081867

[ Test epoch: 13 ]

Test accuarcy: 82.0
Test average loss: 0.005697844439744949
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.890625
Current benign train loss: 0.3717571794986725

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.23915915191173553

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.45322567224502563

Current batch: 300
Current benign train accuracy: 0.875
Current benign train loss: 0.31984952092170715

Total benign train accuarcy: 88.754
Total benign train loss: 129.58686926960945

[ Test epoch: 14 ]

Test accuarcy: 82.09
Test average loss: 0.005357950463891029
Model 


Total benign train accuarcy: 91.766
Total benign train loss: 93.54180114716291

[ Test epoch: 26 ]

Test accuarcy: 88.19
Test average loss: 0.003559704601764679
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.19381745159626007

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.30986693501472473

Current batch: 200
Current benign train accuracy: 0.875
Current benign train loss: 0.4246898889541626

Current batch: 300
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2334621697664261

Total benign train accuarcy: 91.796
Total benign train loss: 92.8858235180378

[ Test epoch: 27 ]

Test accuarcy: 85.9
Test average loss: 0.004439762400090695
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.21917182207107544

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign trai


Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.1544957309961319

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.30632227659225464

Current batch: 200
Current benign train accuracy: 0.9375
Current benign train loss: 0.21215234696865082

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.12886400520801544

Total benign train accuarcy: 92.916
Total benign train loss: 79.66560911387205

[ Test epoch: 40 ]

Test accuarcy: 85.51
Test average loss: 0.004675002235174179
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.1031181663274765

Current batch: 100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09581178426742554

Current batch: 200
Current benign train accuracy: 0.9375
Current benign train loss: 0.19821423292160034

Current batch: 300
Current benign train accuracy: 0.9296875



Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.021529067307710648

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.029315762221813202

Total benign train accuarcy: 98.728
Total benign train loss: 15.09683898370713

[ Test epoch: 53 ]

Test accuarcy: 94.1
Test average loss: 0.0020501150941476228
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.009877849370241165

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.04264254495501518

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00743456743657589

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.05488280951976776

Total benign train accuarcy: 99.0
Total benign train loss: 12.736729582073167

[ Test epoch: 54 ]

Test accuarcy: 94.0
Test average loss: 0.002092910374887288
Model Sav


Total benign train accuarcy: 99.674
Total benign train loss: 4.239061501109973

[ Test epoch: 66 ]

Test accuarcy: 94.14
Test average loss: 0.002374490135535598
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0025706281885504723

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.006711323279887438

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00405547022819519

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0037846616469323635

Total benign train accuarcy: 99.778
Total benign train loss: 3.459391534619499

[ Test epoch: 67 ]

Test accuarcy: 94.27
Test average loss: 0.0023117689160630107
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.0425727516412735

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00486463


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0012848827755078673

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0011981348507106304

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.001200614613480866

Total benign train accuarcy: 99.938
Total benign train loss: 1.6339583697263151

[ Test epoch: 80 ]

Test accuarcy: 94.34
Test average loss: 0.0023784119203686714
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.004686429630964994

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.006621695123612881

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0011008823057636619

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0015586515655741096

Total benign train accuarcy: 99.934
Total benign train loss: 1.509824085893342


Total benign train accuarcy: 99.956
Total benign train loss: 1.2112078108330024

[ Test epoch: 93 ]

Test accuarcy: 94.5
Test average loss: 0.0023559490997344256
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.002166708465665579

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0014826928963884711

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0034116674214601517

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.011461131274700165

Total benign train accuarcy: 99.93
Total benign train loss: 1.4571839662094135

[ Test epoch: 94 ]

Test accuarcy: 94.43
Test average loss: 0.0023614068921655415
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0026273399125784636

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00